# Assignment 2: Cardiovascular Risk
Alex Reese, Scott Ratchford, Ben Fioresi
### Introduction
TODO

Only run this install command on Google Colab, and only once.

In [1]:
# !pip install tensorflow_decision_forests

In [2]:
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv('Heart_failure_dataset.csv', encoding = "utf-8-sig")

# # Separate the Cabin column using '/' as the delimiter
# data[['Cabin_deck', 'Cabin_num', 'Cabin_side']] = data['Cabin'].str.split('/', expand=True)

# # Separate the Name column and keep only the first letter of each name
# data['Name_first'] = data['Name'].str.split().str[0].str[0]
# data['Name_last'] = data['Name'].str.split().str[-1].str[0]

# # Separate the PassengerId column using '_' as the delimiter
# data[['PassengerId_1', 'PassengerId_2']] = data['PassengerId'].str.split('_', expand=True)
# data['PassengerId_1'] = pd.to_numeric(data['PassengerId_1'], errors='coerce')
# data['PassengerId_2'] = pd.to_numeric(data['PassengerId_2'], errors='coerce')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# # Handle missing values: Fill numerical columns with median and categorical columns with mode
# for col in data.columns:
#     if data[col].dtype == 'object':
#         data[col].fillna(data[col].mode()[0], inplace=True)
#     else:
#         data[col].fillna(data[col].median(), inplace=True)

# # Convert boolean columns to integer
# data['CryoSleep'] = data['CryoSleep'].astype(int)
# data['VIP'] = data['VIP'].astype(int)
# data['Transported'] = data['Transported'].astype(int)

# # Integer encode categorical columns
# label_encoders = {}
# for col in ['HomePlanet', 'Cabin_deck', 'Cabin_side', 'Destination']:
#   le = LabelEncoder()
#   data[col] = le.fit_transform(data[col])
#   label_encoders[col] = le

# # Normalize numerical columns
# scaler = StandardScaler()
# numerical_cols = ['Cabin.num', 'Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
# data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

**Only run one of the next two blocks!**

In [4]:
# Drop columns that won't be used for training
# data = data.drop(columns=['PassengerId', 'Name', 'Name_first', 'Name_last', 'Cabin', 'Cabin_num'])

# Split data into training and validation sets                              # stratify insures equal ratio
train_data, test_data = train_test_split(data, test_size=0.2, random_state=32)  # , stratify=data['Age']

# Save the training and test datasets to CSV files
train_data.to_csv('train_data.csv', index=False, encoding="utf-8-sig")
test_data.to_csv('test_data.csv', index=False, encoding="utf-8-sig")

In [5]:
# import tensorflow as tf
# import tensorflow_decision_forests as tfdf

# # Load the training dataset
# dataset_df = pd.read_csv('train_data.csv', encoding = "utf-8-sig")

# # Convert the training data to TensorFlow dataset
# train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(dataset_df, label="Transported")

# # Create a Random Forest model
# model = tfdf.keras.RandomForestModel()
# model.fit(train_ds)

# #model.save('trained_model')

In [6]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# # Load the testing dataset
# dataset_df = pd.read_csv('test_data.csv', encoding = "utf-8-sig")

# # Convert the testing data to TensorFlow dataset
# test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(dataset_df, task=tfdf.keras.Task.CLASSIFICATION, label="Transported")

# # gather predictions from test data
# predictions_proba = model.predict(test_ds).flatten()
# predicted_classes = (predictions_proba > 0.5).astype(int)
# true_labels = test_data["Transported"].values

# # calculate the evaluation metrics
# accuracy = accuracy_score(true_labels, predicted_classes)
# precision = precision_score(true_labels, predicted_classes)
# recall = recall_score(true_labels, predicted_classes)
# f1 = f1_score(true_labels, predicted_classes)
# roc_auc = roc_auc_score(true_labels, predictions_proba)  # Note: Here we use the probabilities, not the thresholded predictions
# conf_matrix = confusion_matrix(true_labels, predicted_classes)

# # print metrics
# print(f"Accuracy: {accuracy}")
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")
# print(f"ROC AUC: {roc_auc}")
# print("Confusion Matrix:")
# print(conf_matrix)